In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rohlik-orders-forecasting-challenge/test_calendar.csv
/kaggle/input/rohlik-orders-forecasting-challenge/solution_example.csv
/kaggle/input/rohlik-orders-forecasting-challenge/train.csv
/kaggle/input/rohlik-orders-forecasting-challenge/test.csv
/kaggle/input/rohlik-orders-forecasting-challenge/train_calendar.csv


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import plotly.express as px

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
train=pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/train.csv')
test=pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/test.csv')
trainc=pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/train_calendar.csv')
testc=pd.read_csv('/kaggle/input/rohlik-orders-forecasting-challenge/test_calendar.csv')

In [7]:
train.head()

,warehouse,date,orders,holiday_name,holiday,shutdown,mini_shutdown,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,user_activity_1,user_activity_2,id
0,Prague_1,2020-12-05,6895.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1722.0,32575.0,Prague_1_2020-12-05
1,Prague_1,2020-12-06,6584.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1688.0,32507.0,Prague_1_2020-12-06
2,Prague_1,2020-12-07,7030.0,NaN,0,0,0,0,0,0,0,0.0,0,0.0,0.0,1696.0,32552.0,Prague_1_2020-12-07
3,Prague_1,2020-12-08,6550.0,NaN,0,0,0,0,0,0,0,0.0,0,0.8,0.0,1681.0,32423.0,Prague_1_2020-12-08
4,Prague_1,2020-12-09,6910.0,NaN,0,0,0,0,0,0,0,0.0,0,0.5,0.0,1704.0,32410.0,Prague_1_2020-12-09


In [8]:
test.head()

,warehouse,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,id
0,Prague_1,2024-03-16,NaN,0,0,0,0,Prague_1_2024-03-16
1,Prague_1,2024-03-17,NaN,0,0,0,0,Prague_1_2024-03-17
2,Prague_1,2024-03-18,NaN,0,0,0,0,Prague_1_2024-03-18
3,Prague_1,2024-03-19,NaN,0,0,0,0,Prague_1_2024-03-19
4,Prague_1,2024-03-20,NaN,0,0,0,0,Prague_1_2024-03-20


In [9]:
trainc.head()

,date,holiday_name,holiday,shutdown,mini_shutdown,warehouse_limited,shops_closed,winter_school_holidays,school_holidays,blackout,mov_change,frankfurt_shutdown,precipitation,snow,warehouse
0,2022-03-09,NaN,0,0,0,0,0,1,0,0,0.0,0,0.0,0.0,Prague_1
1,2019-07-03,NaN,0,0,0,0,0,0,0,0,0.0,0,0.0,0.0,Prague_1
2,2020-05-18,NaN,0,0,0,0,0,0,0,0,0.0,0,0.0,0.0,Prague_1
3,2021-11-09,NaN,0,0,0,0,0,0,0,0,0.0,0,0.0,0.0,Prague_1
4,2023-08-02,NaN,0,0,0,0,0,0,0,0,0.0,0,2.5,0.0,Prague_1


In [10]:
testc.head()

,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,warehouse
0,2024-11-17,Den boje za svobodu a demokracii,1,0,0,0,Prague_1
1,2024-07-04,NaN,0,0,0,0,Prague_1
2,2024-08-04,NaN,0,0,0,0,Prague_1
3,2024-12-01,NaN,0,0,0,0,Prague_1
4,2024-07-24,NaN,0,0,0,0,Prague_1


In [11]:
train.shape

(7340, 18)

In [12]:
trainc.shape

(13307, 15)

In [13]:
train.dtypes

warehouse                  object
date                       object
orders                    float64
holiday_name               object
holiday                     int64
shutdown                    int64
mini_shutdown               int64
shops_closed                int64
winter_school_holidays      int64
school_holidays             int64
blackout                    int64
mov_change                float64
frankfurt_shutdown          int64
precipitation             float64
snow                      float64
user_activity_1           float64
user_activity_2           float64
id                         object
dtype: object

In [14]:
trainc.dtypes

date                       object
holiday_name               object
holiday                     int64
shutdown                    int64
mini_shutdown               int64
warehouse_limited           int64
shops_closed                int64
winter_school_holidays      int64
school_holidays             int64
blackout                    int64
mov_change                float64
frankfurt_shutdown          int64
precipitation             float64
snow                      float64
warehouse                  object
dtype: object

In [15]:
len(train[train.duplicated(['id'])])

0

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7340 entries, 0 to 7339
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   warehouse               7340 non-null   object 
 1   date                    7340 non-null   object 
 2   orders                  7340 non-null   float64
 3   holiday_name            218 non-null    object 
 4   holiday                 7340 non-null   int64  
 5   shutdown                7340 non-null   int64  
 6   mini_shutdown           7340 non-null   int64  
 7   shops_closed            7340 non-null   int64  
 8   winter_school_holidays  7340 non-null   int64  
 9   school_holidays         7340 non-null   int64  
 10  blackout                7340 non-null   int64  
 11  mov_change              7340 non-null   float64
 12  frankfurt_shutdown      7340 non-null   int64  
 13  precipitation           7070 non-null   float64
 14  snow                    7070 non-null   

In [17]:
## concatenate test and training dataset 
base_features=test.drop(columns='id').columns
test_id=test.id

In [19]:
all_df=pd.concat([train[base_features],train['orders']],axis=1)
all_df=pd.concat([all_df,test[base_features]],sort=False).reset_index(drop=True)

In [20]:
all_df.columns

Index(['warehouse', 'date', 'holiday_name', 'holiday', 'shops_closed',
       'winter_school_holidays', 'school_holidays', 'orders'],
      dtype='object')

In [21]:
date_col = 'date'
all_df[date_col] = pd.to_datetime(all_df[date_col], errors='coerce')
all_df[date_col + "_year"] = all_df[date_col].dt.year.fillna(-1)
all_df[date_col + "_month"] = all_df[date_col].dt.month.fillna(-1)
all_df[date_col + "_day"] = all_df[date_col].dt.day.fillna(-1)
all_df[date_col + "_day_of_week"] = all_df[date_col].dt.dayofweek.fillna(-1)
all_df.drop(date_col, axis=1, inplace=True)

In [22]:
all_df.columns

Index(['warehouse', 'holiday_name', 'holiday', 'shops_closed',
       'winter_school_holidays', 'school_holidays', 'orders', 'date_year',
       'date_month', 'date_day', 'date_day_of_week'],
      dtype='object')

In [23]:
all_df.holiday_name.fillna('None',inplace=True)

In [24]:
all_df.warehouse.value_counts(normalize=True)

warehouse
Prague_1       0.162078
Brno_1         0.162078
Prague_2       0.162078
Prague_3       0.162078
Budapest_1     0.156521
Munich_1       0.107664
Frankfurt_1    0.087502
Name: proportion, dtype: float64

In [25]:
all_df.holiday_name.value_counts(normalize=True)

holiday_name
None                                                           0.969626
International womens day                                       0.003360
Christmas Eve                                                  0.002973
2nd Christmas Day                                              0.002068
New Years Day                                                  0.002068
Good Friday                                                    0.002068
Easter Monday                                                  0.002068
Labour Day                                                     0.002068
Den osvobozeni                                                 0.002068
Den ceske statnosti                                            0.001551
Den boje za svobodu a demokracii                               0.001551
Den vzniku samostatneho ceskoslovenskeho statu                 0.001551
Jan Hus                                                        0.001551
Cyrila a Metodej                                   

In [26]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder


In [27]:
enc = OneHotEncoder(sparse=False)
holiday_encoded = enc.fit_transform(all_df[['holiday_name']])
encoded_df = pd.DataFrame(holiday_encoded, columns=enc.get_feature_names_out(['holiday_name']))
all_df = pd.concat([all_df, encoded_df], axis=1)
all_df.drop('holiday_name', axis=1, inplace=True)

In [28]:
all_df.columns

Index(['warehouse', 'holiday', 'shops_closed', 'winter_school_holidays',
       'school_holidays', 'orders', 'date_year', 'date_month', 'date_day',
       'date_day_of_week',
       'holiday_name_1848 Revolution Memorial Day (Extra holiday)',
       'holiday_name_2nd Christmas Day',
       'holiday_name_All Saints' Day Holiday', 'holiday_name_Christmas Eve',
       'holiday_name_Cyrila a Metodej', 'holiday_name_Day of National Unity',
       'holiday_name_Den boje za svobodu a demokracii',
       'holiday_name_Den ceske statnosti', 'holiday_name_Den osvobozeni',
       'holiday_name_Den vzniku samostatneho ceskoslovenskeho statu',
       'holiday_name_Easter Monday', 'holiday_name_Good Friday',
       'holiday_name_Independent Hungary Day',
       'holiday_name_International womens day', 'holiday_name_Jan Hus',
       'holiday_name_Labour Day',
       'holiday_name_Memorial Day for the Martyrs of Arad',
       'holiday_name_Memorial Day for the Victims of the Communist Dictatorships',


In [29]:
le = LabelEncoder()
all_df['warehouse'] = le.fit_transform(all_df['warehouse'])

In [30]:
all_df.columns

Index(['warehouse', 'holiday', 'shops_closed', 'winter_school_holidays',
       'school_holidays', 'orders', 'date_year', 'date_month', 'date_day',
       'date_day_of_week',
       'holiday_name_1848 Revolution Memorial Day (Extra holiday)',
       'holiday_name_2nd Christmas Day',
       'holiday_name_All Saints' Day Holiday', 'holiday_name_Christmas Eve',
       'holiday_name_Cyrila a Metodej', 'holiday_name_Day of National Unity',
       'holiday_name_Den boje za svobodu a demokracii',
       'holiday_name_Den ceske statnosti', 'holiday_name_Den osvobozeni',
       'holiday_name_Den vzniku samostatneho ceskoslovenskeho statu',
       'holiday_name_Easter Monday', 'holiday_name_Good Friday',
       'holiday_name_Independent Hungary Day',
       'holiday_name_International womens day', 'holiday_name_Jan Hus',
       'holiday_name_Labour Day',
       'holiday_name_Memorial Day for the Martyrs of Arad',
       'holiday_name_Memorial Day for the Victims of the Communist Dictatorships',


In [31]:
all_df.warehouse.value_counts(normalize=True)

warehouse
4    0.162078
0    0.162078
5    0.162078
6    0.162078
1    0.156521
3    0.107664
2    0.087502
Name: proportion, dtype: float64

In [33]:
##  convert back
train_df=all_df[~all_df['orders'].isnull()]
test_df=all_df[all_df['orders'].isnull()]

In [44]:
from sklearn.model_selection import train_test_split

In [45]:
X=train_df.drop(columns=['orders'])
y=train_df['orders']

In [46]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=290)

In [47]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.ensemble import VotingRegressor, HistGradientBoostingRegressor, GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [49]:
xgb_model = XGBRegressor()
lgb_model = LGBMRegressor()
cat_model = CatBoostRegressor(silent=True)
hgb_model = HistGradientBoostingRegressor()
gb_model = GradientBoostingRegressor()
rf_model = RandomForestRegressor()

voting_reg=VotingRegressor(
    estimators=[
        ('xgb',xgb_model),
        ('lgb',lgb_model),
        ('cat', cat_model),
        ('hgb', hgb_model),
        ('gb', gb_model),
        ('rf', rf_model)
    ]
)

voting_reg.fit(X_train, y_train)

y_pred = voting_reg.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'Voting Regressor MAPE: {mape:.4f}')

test_df = test_df.drop(columns=['orders'])
submit_pred = voting_reg.predict(test_df)

submission = pd.DataFrame({
    'id': test_id,
    'Target': submit_pred
})


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000958 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75
[LightGBM] [Info] Number of data points in the train set: 5872, number of used features: 10
[LightGBM] [Info] Start training from score 5547.934946
Voting Regressor MAPE: 0.0437


In [54]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split, cross_val_score, KFold
xgb_params = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0]
}

lgb_params = {
    'n_estimators': [150, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'num_leaves': [31, 50, 100],
    'subsample': [0.8, 1.0]
}

cat_params = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'depth': [3, 5, 7],
    'subsample': [0.8, 1.0]
}

xgb_random = RandomizedSearchCV(XGBRegressor(), param_distributions=xgb_params, n_iter=15, scoring='neg_mean_absolute_percentage_error', cv=3, random_state=42)
xgb_random.fit(X_train, y_train)
xgb_best = xgb_random.best_estimator_

lgb_random = RandomizedSearchCV(LGBMRegressor(), param_distributions=lgb_params, n_iter=15, scoring='neg_mean_absolute_percentage_error', cv=3, random_state=42)
lgb_random.fit(X_train, y_train)
lgb_best = lgb_random.best_estimator_

cat_random = RandomizedSearchCV(CatBoostRegressor(silent=True), param_distributions=cat_params, n_iter=15, scoring='neg_mean_absolute_percentage_error', cv=3, random_state=42)
cat_random.fit(X_train, y_train)
cat_best = cat_random.best_estimator_

hgb_model = HistGradientBoostingRegressor()
gb_model = GradientBoostingRegressor()
rf_model = RandomForestRegressor()

voting_reg = VotingRegressor(
    estimators=[
        ('xgb', xgb_best),
        ('lgb', lgb_best),
        ('cat', cat_best),
        ('hgb', hgb_model),
        ('gb', gb_model),
        ('rf', rf_model)
    ]
)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(voting_reg, X, y, cv=kf, scoring='neg_mean_absolute_percentage_error')

mean_cv_score = np.mean(-cv_scores)
print(f'Mean CV MAPE: {mean_cv_score:.4f}')

voting_reg.fit(X, y)

submission.to_csv('submission.csv', index=False)
print(submission.head())

y_pred = voting_reg.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f'Voting Regressor MAPE after hyperparameter tuning: {mape:.4f}')

submit_pred = voting_reg.predict(test_df)

submission = pd.DataFrame({
    'id': test_id,
    'Target': submit_pred
})


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000633 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 75
[LightGBM] [Info] Number of data points in the train set: 3914, number of used features: 10
[LightGBM] [Info] Start training from score 5576.110373
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000732 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 73
[LightGBM] [Info] Number of data points in the train set: 3915, number of used features: 9
[LightGBM] [Info] Start training from score 5552.889655
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of tes

In [57]:
submission.to_csv('submission_Rohlik_again.csv', index=False)
print(submission.head())

                    id        Target
0  Prague_1_2024-03-16  10440.433960
1  Prague_1_2024-03-17  10268.576850
2  Prague_1_2024-03-18   9783.255513
3  Prague_1_2024-03-19   9539.285718
4  Prague_1_2024-03-20   9515.779746
